In [59]:
import os
import pandas as pd
import optuna
import numpy as np
import openml
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido
import plotly
import scipy.stats as stats
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from optuna.samplers import TPESampler, CmaEsSampler
from xgboost import XGBClassifier
from plotly.io import show
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier

In [60]:
dataset_ids = [
    1590,   # Adult
    1510,   # Breast Cancer Wisconsin (Diagnostic)
    1461,   # Bank Marketing
    24,     # Mushroom
    40945   # Titanic
]

In [61]:
def fetch_and_prepare(openml_id):
    global y
    global X
    dataset = openml.datasets.get_dataset(openml_id)
    print(f">>> {dataset.name} (ID: {openml_id})")
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')
    for col in X.select_dtypes(include=['category', 'object']):
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    #scaler = MinMaxScaler()
    #X_imputed = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)
    if y.dtype == 'category':
        y = LabelEncoder().fit_transform(y)
    zbior = X_imputed['Target'] = y
    return X_imputed

In [62]:
zbior = fetch_and_prepare(1590)

>>> adult (ID: 1590)


In [64]:
X_train, X_test, y_train, y_test = train_test_split(zbior.iloc[:, :-1], zbior['Target'], test_size=0.30, random_state=42)

def objective(trial):

    ccp_alpha = trial.suggest_float("ccp_alpha", 0.0, 0.008, log=False)
    max_depth = trial.suggest_int("max_depth", 12, 27)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 4, 42)
    min_samples_split = trial.suggest_int("min_samples_split", 5, 49)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    wyniki = []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        X_train_fold = X_train_fold.reset_index(drop=True)
        X_test_fold = X_test_fold.reset_index(drop=True)
        y_train_fold = y_train_fold.reset_index(drop=True)
        y_test_fold = y_test_fold.reset_index(drop=True)

        X_train_fold = list(zip(*[X_train_fold[col] for col in X_train_fold]))
        X_test_fold = list(zip(*[X_test_fold[col] for col in X_test_fold]))

        Drzewo_losowe = DecisionTreeClassifier(ccp_alpha = ccp_alpha, max_depth = max_depth, min_samples_leaf = min_samples_leaf,
                                           min_samples_split = min_samples_split)

        model = Drzewo_losowe.fit(X_train_fold, y_train_fold)


        y_pred_proba = model.predict_proba(X_test_fold)[:, 1]
        auc = roc_auc_score(y_test_fold, y_pred_proba)
                                            
        wyniki.append(auc)

    rezultat = np.mean(wyniki)
    return rezultat

In [65]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
wynik = study.optimize(objective, n_trials=10)

[I 2025-11-12 21:42:19,988] A new study created in memory with name: no-name-184fb632-0947-43ef-bb32-4f74b5aeaba4
[I 2025-11-12 21:42:20,961] Trial 0 finished with value: 0.8861581750948997 and parameters: {'ccp_alpha': 0.0013470831517502654, 'max_depth': 15, 'min_samples_leaf': 24, 'min_samples_split': 33}. Best is trial 0 with value: 0.8861581750948997.
[I 2025-11-12 21:42:21,947] Trial 1 finished with value: 0.889477485362615 and parameters: {'ccp_alpha': 0.0007129836072399716, 'max_depth': 26, 'min_samples_leaf': 28, 'min_samples_split': 35}. Best is trial 1 with value: 0.889477485362615.
[I 2025-11-12 21:42:22,919] Trial 2 finished with value: 0.8430092604721684 and parameters: {'ccp_alpha': 0.005152894704320677, 'max_depth': 17, 'min_samples_leaf': 31, 'min_samples_split': 38}. Best is trial 1 with value: 0.889477485362615.
[I 2025-11-12 21:42:23,841] Trial 3 finished with value: 0.8430092604721684 and parameters: {'ccp_alpha': 0.006594040697990913, 'max_depth': 13, 'min_samples_

In [66]:
study.best_params

{'ccp_alpha': 0.00019474627238946863,
 'max_depth': 25,
 'min_samples_leaf': 7,
 'min_samples_split': 27}

In [57]:
#Defaultowe
Drzewo_losowe = DecisionTreeClassifier(ccp_alpha =  0.01, max_depth =  30, min_samples_leaf = 7,
                                           min_samples_split = 20)
model = Drzewo_losowe.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.8499939349546007


In [58]:
#Optymalne
Drzewo_losowe = DecisionTreeClassifier(ccp_alpha = 0.00019474627238946863, max_depth = 15, min_samples_leaf = 13,
                                           min_samples_split = 43)
model = Drzewo_losowe.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.892479812249101
